In [1]:
path = "./20210425/0003.txt"
with open(path) as f:
    texts = f.read().split("\n")
    texts = [ row.split() for row in texts]
    texts = [[int(x) for x in xs] for xs in texts]
    
def texts_to_stp(texts):
    SI, SJ = [x for x in texts[0]]
    ts_list = texts[1:51]
    ps_list = texts[51:101]

    return SI, SJ, ts_list, ps_list

SI, SJ, ts_list, ps_list = texts_to_stp(texts)

In [5]:
"""
スコアを上げるやり方？
高速化?
1つ戻るのではなく大きく戻る。
"""

import time
start_time = time.time()

N=50

def row_text_to_ints(row):
    return [int(x) for x in row.split()]

fuchi = [-1 for i in range(N+2)]

def add_fuchi(hoges_list):
    hoges_list = [[-1] + hoges + [-1] for hoges in hoges_list]
    hoges_list = [fuchi.copy()] + hoges_list + [fuchi.copy()]
    return hoges_list

def get_input():
    SI, SJ = row_text_to_ints(input())
    # 淵に-1を追加
    ts_list = []
    for _ in range(N):
        ts_list.append(row_text_to_ints(input()))
    
    ps_list = []
    for _ in range(N):
        ps_list.append(row_text_to_ints(input()))
        
    return SI, SJ, ts_list, ps_list

si, sj, ts_list, ps_list = texts_to_stp(texts)
# si, sj, ts_list, ps_list = get_input()

def get_tp_ratio():
    tp_dict ={}
    for ts, ps in zip(ts_list,ps_list):
        for t,p in zip(ts, ps):
            if t in tp_dict:
                tp_dict[t].append(p)
            else:
                tp_dict[t] = [p]
    tp_ratio = {}
    for t, ps in tp_dict.items():
        if len(ps) == 1:
            tp_ratio[t] = {ps[0]: 1.0}
        else:
            goukei = sum(ps) + 1
            tp_ratio[t] = {ps[0]: ps[0]/goukei, ps[1]: ps[1]/goukei}
    return tp_ratio

tp_ratio = get_tp_ratio()
        
ts_list = add_fuchi(ts_list)
ps_list = add_fuchi(ps_list)
si += 1
sj += 1

import random

# 移動できる方にランダムに移動する
# 淵を-1とする
passed = [-1]

passed.append(ts_list[si][sj])

def get_next_tiles(i, j, score):
    ut = ts_list[i-1][j]
    dt = ts_list[i+1][j]
    lt = ts_list[i][j-1]
    rt = ts_list[i][j+1]

    # tile, i, j をタプルで保存
    tiles = []
    if ut not in passed:
        next_p = ps_list[i-1][j]
        tiles.append((ut, i-1, j, score + next_p, "U", next_p))
    if dt not in passed:
        next_p = ps_list[i+1][j]
        tiles.append((dt, i+1, j, score +next_p,  "D", next_p))
    if lt not in passed:
        next_p = ps_list[i][j-1]
        tiles.append((lt, i, j-1,  score + next_p, "L", next_p))
    if rt not in passed:
        next_p = ps_list[i][j+1]
        tiles.append((rt, i, j+1,  score + next_p, "R", next_p))
    # 候補をスコアが高い順でソート
    if tiles:
        sorted(tiles, key=lambda x: -tp_ratio[x[0]][x[5]])
    return tiles

best_tiles = []
# taileID, i, j, その時点でのポイント、コマンドを保存する。最初はコマンドなし
best_tiles.append((ts_list[si][sj],si, sj, ps_list[si][sj], ""))
                  
not_passed_candidates_list = []
now_i, now_j = si, sj
now_score = ps_list[now_i][now_j]
max_score = 0
max_score_commands = ""
commands=""
is_end = False


while True:
#     print("while start")
    elapsed_time = time.time() - start_time
    if elapsed_time > 1.90:
        break
        
    candidates = get_next_tiles(now_i, now_j, now_score)
#     print("now_score", now_score)
    if len(candidates) == 0:
#         print("戻る")
        # 行き先がないため戻る
        # この時点でのスコアがこれまでで最大なら、スコアとコマンドを保存しておく。
        if now_score > max_score:
            max_score = now_score
            max_score_commands = commands
        
        # 戻る際には、他の候補がありえたところまで戻る
        while len(not_passed_candidates_list) >0:
            elapsed_time = time.time() - start_time
            if elapsed_time > 1.90:
                is_end=True
                break
            
            back_count = 1
            if len(commands) > 100:
                back_count = random.randint(1,3)
            elif len(commands) > 500:
                back_count = random.randint(1,5)
            
            # 戻る際には、 passed,  commandを修正する必要。
            # score、位置はcandidatesに保存されておりそちらを参照するため、修正しなくて良い。
            passed = passed[:-back_count]
            commands = commands[:-back_count]
            if back_count >1:
                not_passed_candidates_list = not_passed_candidates_list[:-(back_count-1)]
                
            # 最後尾を取り出す
            previous_candidates = not_passed_candidates_list.pop()
#             print("pc",previous_candidates)
            if len(previous_candidates) > 0:
                # 候補があったらそれらをcandidatesにして元の動作を始める
                candidates = previous_candidates
                break
            # candidatesもなく、not_passed_candidates_listが尽きたなら、最大のやつをprintして終了。
            # TODO: 怪しい
            if len(not_passed_candidates_list) == 0:
                is_end = True
                
            # 1つ前に他の候補がなければもう1つ前に戻るしかない
#             print("continue")
            continue
            
    if is_end:
        break

    # candidatesの1つ目以降は行かないリスト。後で行くかも。
    not_passed_candidates_list.append(candidates[1:])

    # 次行く先はcandidatesの先頭
    next_tile = candidates[0]
    passed.append(next_tile[0])
    now_i, now_j = next_tile[1], next_tile[2]
    now_score = next_tile[3]
    commands += next_tile[4]

# print("last")
print(max_score_commands)

URURURURUUUULLULULUUUUUUUUUUUUUUUUUUURURRRDDRDRRUURURDRDDRDDLLDLULDDDLLUULUULLDDLDDDDRRRRRDDLDLLDDDRDRDDDDDRRUUUURUUUURUUUUUURRURRDRDRRUUUURUUUUURDRDRDDRRDRUURRDDDDDLLULDLLDDDRRDDDDLDLDLULLULLUUUULDLDLLDDDDDDLDDDDRDDDDLULLDDDDLLDDDDDDDDLLLDLDDDDRRUURURDRRDDDDLDDDDRRRRUUURRDDRRDRURRURUUUULUUUULUULLDDDDDDDLLULULUUUUUUUUULUURDRURURUULUUUUURURURRUUURDRRDDDDRRDDDDLDDDDDLDDDDDRRDDDLDLDRRDRRUUUURURRDDDRRUUUUUUUUUUULLULDLDLLUURUURUUUULUUUUULUURUURURUUURRDRUUURRURUULUUURRUURRRRRDRDDRRURURURDDRDDDDDLULDLDDDDDDLLUULUULULULDDLDDLLULDDDLDDRRDDLDLLDLLLDLDDDDRDDRURURDRURURURUUUURURDDRDDLDDDDLDDDDDLLDDDDDDDDDDDRDLLDLLLLDLDDDDRURDRRRUURRURDDRRURDRUURRURDRRRDRRUUULULUUUUUUULULLUUUUUUULLUULUUURURUULLUURUURRRDDDDDDDLDDDRRDDRDRUUUUUURUURUULLLUURURULLUULUURRDRUUUURUUL


In [7]:
"""
スコアを上げるやり方？
高速化?
タイルのうち、大きい方を踏むように優先する。
なぜなら、低い方を踏んでしまうと、高い方は一生踏めないから。
本当か？単純にスコアが大きいかどうかで選ぶ方がいいかも知れない？

"""

import time
start_time = time.time()

N=50

def row_text_to_ints(row):
    return [int(x) for x in row.split()]

fuchi = [-1 for i in range(N+2)]

def add_fuchi(hoges_list):
    hoges_list = [[-1] + hoges + [-1] for hoges in hoges_list]
    hoges_list = [fuchi.copy()] + hoges_list + [fuchi.copy()]
    return hoges_list

def get_input():
    SI, SJ = row_text_to_ints(input())
    # 淵に-1を追加
    ts_list = []
    for _ in range(N):
        ts_list.append(row_text_to_ints(input()))
    
    ps_list = []
    for _ in range(N):
        ps_list.append(row_text_to_ints(input()))
        
    return SI, SJ, ts_list, ps_list

si, sj, ts_list, ps_list = texts_to_stp(texts)
# si, sj, ts_list, ps_list = get_input()

def get_tp_ratio():
    tp_dict ={}
    for ts, ps in zip(ts_list,ps_list):
        for t,p in zip(ts, ps):
            if t in tp_dict:
                tp_dict[t].append(p)
            else:
                tp_dict[t] = [p]
    tp_ratio = {}
    for t, ps in tp_dict.items():
        if len(ps) == 1:
            tp_ratio[t] = {ps[0]: 1.0}
        else:
            goukei = sum(ps) + 1
            tp_ratio[t] = {ps[0]: ps[0]/goukei, ps[1]: ps[1]/goukei}
    return tp_ratio

tp_ratio = get_tp_ratio()
        
ts_list = add_fuchi(ts_list)
ps_list = add_fuchi(ps_list)
si += 1
sj += 1

import random

# 移動できる方にランダムに移動する
# 淵を-1とする
passed = [-1]

passed.append(ts_list[si][sj])

def get_next_tiles(i, j, score):
    ut = ts_list[i-1][j]
    dt = ts_list[i+1][j]
    lt = ts_list[i][j-1]
    rt = ts_list[i][j+1]

    # tile, i, j をタプルで保存
    tiles = []
    if ut not in passed:
        next_p = ps_list[i-1][j]
        tiles.append((ut, i-1, j, score + next_p, "U", next_p))
    if dt not in passed:
        next_p = ps_list[i+1][j]
        tiles.append((dt, i+1, j, score +next_p,  "D", next_p))
    if lt not in passed:
        next_p = ps_list[i][j-1]
        tiles.append((lt, i, j-1,  score + next_p, "L", next_p))
    if rt not in passed:
        next_p = ps_list[i][j+1]
        tiles.append((rt, i, j+1,  score + next_p, "R", next_p))
    # 候補をスコアが高い順でソート
    if tiles:
        sorted(tiles, key=lambda x: -tp_ratio[x[0]][x[5]])
    return tiles

best_tiles = []
# taileID, i, j, その時点でのポイント、コマンドを保存する。最初はコマンドなし
best_tiles.append((ts_list[si][sj],si, sj, ps_list[si][sj], ""))
                  
not_passed_candidates_list = []
now_i, now_j = si, sj
now_score = ps_list[now_i][now_j]
max_score = 0
max_score_commands = ""
commands=""
is_end = False


while True:
#     print("while start")
    elapsed_time = time.time() - start_time
    if elapsed_time > 1.9:
        break
        
    candidates = get_next_tiles(now_i, now_j, now_score)
#     print("now_score", now_score)
    if len(candidates) == 0:
#         print("戻る")
        # 行き先がないため戻る
        # この時点でのスコアがこれまでで最大なら、スコアとコマンドを保存しておく。
        if now_score > max_score:
            max_score = now_score
            max_score_commands = commands
        
        # 戻る際には、他の候補がありえたところまで戻る
        while len(not_passed_candidates_list) >0:
            elapsed_time = time.time() - start_time
            if elapsed_time > 1.9:
                is_end=True
                break

            # 戻る際には、 passed,  commandを修正する必要。
            # score、位置はcandidatesに保存されておりそちらを参照するため、修正しなくて良い。
            passed = passed[:-1]
            commands = commands[:-1]
            
            # 最後尾を取り出す
            previous_candidates = not_passed_candidates_list.pop()
#             print("pc",previous_candidates)
            if len(previous_candidates) > 0:
                # 候補があったらそれらをcandidatesにして元の動作を始める
                candidates = previous_candidates
                break
            # candidatesもなく、not_passed_candidates_listが尽きたなら、最大のやつをprintして終了。
            # TODO: 怪しい
            if len(not_passed_candidates_list) == 0:
                is_end = True
                
            # 1つ前に他の候補がなければもう1つ前に戻るしかない
#             print("continue")
            continue
            
    if is_end:
        break

    # candidatesの1つ目以降は行かないリスト。後で行くかも。
    not_passed_candidates_list.append(candidates[1:])

    # 次行く先はcandidatesの先頭
    next_tile = candidates[0]
    passed.append(next_tile[0])
    now_i, now_j = next_tile[1], next_tile[2]
    now_score = next_tile[3]
    commands += next_tile[4]

# print("last")
print(max_score_commands)

URURURURUUUULLULULUUUUUUUUUUUUUUUUUUURURRRDDRDRRUURURDRDDRDDLLDLULDDDLLUULUULLDDLDDDDRRRRRDDLDLLDDDDDDRDRUUUURURUURUURUUURRURRDRDRURUUURUUUUURDRDRDDRRDRUURRDDDDDLLULDLLDDDLULDDLDLDDLLDLDDLLDLDDDDLLDDRDRDRUUURUURUURDRDDRRDDDDDLDDDDDDDDLULUULUUULULDLDDDLLUULDLUULLLDLDDLLDDRRDDDRDRDDDDRURRDDDDDDDRRRRUUULULUUUURURUUUURDRDRDDDDDLDDRRDDRRUUUURUUUULUURUUURUUUUUURUURUURURUUUUUUULLUUUURURDDRDDDDRRUUUUUUURRDRUUURRURUULUUURRUURRRRRDRDDLLDLLDDRRRDRDRURURURUUURRDDDDDLDLDLDDDDLLUULULLULLDDLLULDDDLDDLULLDDRDDDLDLLDDDDDDDDLDLLDDLDDDLDDDRDDDLDDRDRRUUUURURRDDDDDRDDDLDDRRURUUURRDDDDDRRRURURURDRUURRURDDRURRDRRUUULULUUUUUUUULLDDDLDDDLDLLULUUUUUULLDDLDDDDDDLLUUUUUUUUUUUULLUUULULLUUURURUURURDDDDDDDDRURURUUUUUURUURRURURRRRDDDDDDDDDLDLULUULLDDDDDDRRDRUURRRRDRUUUUULUURURRUULLLUURURRULLLLUUUURRDRUUUURUUL


In [2]:
# def answers_list_to_str(answers_list):
#     answers_str_list = [" ".join([str(answer) for answer in answers])  for answers in answers_list]
#     answers_str = "\n".join(answers_str_list)
#     return answers_str

# def row_text_to_ints(row):
#     return [int(x) for x in row.split()]

# def get_input():
#     N=50
#     SI, SJ = row_text_to_ints(input())
#     ts_list = []
#     ts_list.append(fuchi.copy())    
#     for _ in range(N):
#         ts_list.append([-1] + row_text_to_ints(input())+[-1])
#     ts_list.append(fuchi.copy())    

#     ps_list = []
#     ps_list.append(fuchi.copy())    
#     for _ in range(N):
#         ps_list.append([-1] + row_text_to_ints(input())[-1])
#     ps_list.append(fuchi.copy())    

#     return SI, SJ, ts_list, ps_list

SI, SJ, ts_list, ps_list = get_input()

NameError: name 'get_input' is not defined

In [4]:
[-1] +[1,2,3] + [-1]

[-1, 1, 2, 3, -1]

In [5]:
[1,2,3].copy()

[1, 2, 3]

In [19]:
len(add_fuchi(ts_list)[0])

52

In [15]:
[len(ts) for ts in add_fuchi(ts_list)]

TypeError: object of type 'int' has no len()

In [23]:
# まずは何もしない
N=50
def answers_list_to_str(answers_list):
    answers_str_list = [" ".join([str(answer) for answer in answers])  for answers in answers_list]
    answers_str = "\n".join(answers_str_list)
    return answers_str

def row_text_to_ints(row):
    return [int(x) for x in row.split()]

fuchi = [-1 for i in range(N+2)]

def get_input():
    SI, SJ = row_text_to_ints(input())
    # 淵に-1を追加
    ts_list = []
    for _ in range(N):
        ts_list.append([-1] + row_text_to_ints(input()) + [-1])
    
    ps_list = []
    for _ in range(N):
        ps_list.append([-1] + row_text_to_ints(input())[-1])
        
    return SI, SJ, ts_list, ps_list

si, sj, ts_list, ps_list = texts_to_stp(texts)
# SI, SJ, ts_list, ps_list = get_input()

ts_list = add_fuchi(ts_list)
ps_list = add_fuchi(ps_list)
si += 1
sj += 1


In [4]:
# N=50
# def answers_list_to_str(answers_list):
#     answers_str_list = [" ".join([str(answer) for answer in answers])  for answers in answers_list]
#     answers_str = "\n".join(answers_str_list)
#     return answers_str

# def row_text_to_ints(row):
#     return [int(x) for x in row.split()]

# fuchi = [-1 for i in range(N+2)]

# def add_fuchi(hoges_list):
#     hoges_list = [[-1] + hoges + [-1] for hoges in hoges_list]
#     hoges_list = [fuchi.copy()] + hoges_list + [fuchi.copy()]
#     return hoges_list

# def get_input():
#     SI, SJ = row_text_to_ints(input())
#     # 淵に-1を追加
#     ts_list = []
#     for _ in range(N):
#         ts_list.append(row_text_to_ints(input()))
    
#     ps_list = []
#     for _ in range(N):
#         ps_list.append(row_text_to_ints(input()))
        
#     return SI, SJ, ts_list, ps_list

# si, sj, ts_list, ps_list = texts_to_stp(texts)
# # si, sj, ts_list, ps_list = get_input()

# ts_list = add_fuchi(ts_list)
# ps_list = add_fuchi(ps_list)
# si += 1
# sj += 1

# import random

# # 移動できる方にランダムに移動する
# # 淵を-1とする
# passed = set([-1])

# passed.add(ts_list[si][sj])

# def get_next_tiles(i, j):
#     ut = ts_list[i-1][j]
#     dt = ts_list[i+1][j]
#     lt = ts_list[i][j-1]
#     rt = ts_list[i][j+1]

#     # tile, i, j をタプルで保存
#     tiles = []
#     if ut not in passed:
#         tiles.append((ut, i-1, j, "U"))
#     if dt not in passed:
#         tiles.append((dt, i+1, j, "D"))
#     if lt not in passed:
#         tiles.append((lt, i, j-1, "L"))
#     if rt not in passed:
#         tiles.append((rt, i, j+1, "R"))

#     return tiles

# now_i, now_j = si, sj
# commands = ""

# while True:
#     candidates = get_next_tiles(now_i, now_j)
#     if len(candidates) == 0:
#         # 行き先がないため終了
#         break
#     # 次行く先
#     next_tile = random.choice(candidates)
#     passed.add(next_tile[0])
#     now_i, now_j = next_tile[1], next_tile[2]
#     commands += next_tile[3]
# print(commands)

LDLLLDDRRDDLLLULLDLLDDRDRDDDRURRUUURD


In [4]:
"""
止まってしまったら、引き戻せば良いのでは。
時間も考慮すること
"""


import time
start_time = time.time()

N=50

def row_text_to_ints(row):
    return [int(x) for x in row.split()]

fuchi = [-1 for i in range(N+2)]

def add_fuchi(hoges_list):
    hoges_list = [[-1] + hoges + [-1] for hoges in hoges_list]
    hoges_list = [fuchi.copy()] + hoges_list + [fuchi.copy()]
    return hoges_list

def get_input():
    SI, SJ = row_text_to_ints(input())
    # 淵に-1を追加
    ts_list = []
    for _ in range(N):
        ts_list.append(row_text_to_ints(input()))
    
    ps_list = []
    for _ in range(N):
        ps_list.append(row_text_to_ints(input()))
        
    return SI, SJ, ts_list, ps_list

si, sj, ts_list, ps_list = texts_to_stp(texts)
# si, sj, ts_list, ps_list = get_input()



ts_list = add_fuchi(ts_list)
ps_list = add_fuchi(ps_list)
si += 1
sj += 1

import random

# 移動できる方にランダムに移動する
# 淵を-1とする
passed = [-1]

passed.append(ts_list[si][sj])

def get_next_tiles(i, j, score):
    ut = ts_list[i-1][j]
    dt = ts_list[i+1][j]
    lt = ts_list[i][j-1]
    rt = ts_list[i][j+1]

    # tile, i, j をタプルで保存
    tiles = []
    if ut not in passed:
        tiles.append((ut, i-1, j, score +ps_list[i-1][j], "U"))
    if dt not in passed:
        tiles.append((dt, i+1, j, score +ps_list[i+1][j],  "D"))
    if lt not in passed:
        tiles.append((lt, i, j-1,  score +ps_list[i][j-1], "L"))
    if rt not in passed:
        tiles.append((rt, i, j+1,  score +ps_list[i][j+1], "R"))
    # 候補をシャッフル
    random.shuffle(tiles)
    return tiles

best_tiles = []
# taileID, i, j, その時点でのポイント、コマンドを保存する。最初はコマンドなし
best_tiles.append((ts_list[si][sj],si, sj, ps_list[si][sj], ""))
                  
not_passed_candidates_list = []
now_i, now_j = si, sj
now_score = ps_list[now_i][now_j]
max_score = 0
max_score_commands = ""
commands=""
is_end = False


while True:
#     print("while start")
    elapsed_time = time.time() - start_time
    if elapsed_time > 1.9:
        break
        
    candidates = get_next_tiles(now_i, now_j, now_score)
#     print("now_score", now_score)
    if len(candidates) == 0:
#         print("戻る")
        # 行き先がないため戻る
        # この時点でのスコアがこれまでで最大なら、スコアとコマンドを保存しておく。
        if now_score > max_score:
            max_score = now_score
            max_score_commands = commands
        
        # 戻る際には、他の候補がありえたところまで戻る
        while len(not_passed_candidates_list) >0:
            elapsed_time = time.time() - start_time
            if elapsed_time > 1.9:
                is_end=True
                break

            # 戻る際には、 passed,  commandを修正する必要。
            # score、位置はcandidatesに保存されておりそちらを参照するため、修正しなくて良い。
            passed = passed[:-1]
            commands = commands[:-1]
            
            # 最後尾を取り出す
            previous_candidates = not_passed_candidates_list.pop()
#             print("pc",previous_candidates)
            if len(previous_candidates) > 0:
                # 候補があったらそれらをcandidatesにして元の動作を始める
                candidates = previous_candidates
                break
            # candidatesもなく、not_passed_candidates_listが尽きたなら、最大のやつをprintして終了。
            # TODO: 怪しい
            if len(not_passed_candidates_list) == 0:
                is_end = True
                
            # 1つ前に他の候補がなければもう1つ前に戻るしかない
#             print("continue")
            continue
            
    if is_end:
        break

    # candidatesの1つ目以降は行かないリスト。後で行くかも。
    not_passed_candidates_list.append(candidates[1:])

    # 次行く先はcandidatesの先頭
    next_tile = candidates[0]
    passed.append(next_tile[0])
    now_i, now_j = next_tile[1], next_tile[2]
    now_score = next_tile[3]
    commands += next_tile[4]

# print("last")
print(max_score_commands)

DLDLLUUUULDLUULULULLULLLDLLDLDLLUUULDLLULLDDLDDLDLDDDDDDRDLDLLDDDRDDDDDDDDRRURRUULLULURURRURURURURRRUULULUULULDLULUUURRURRDRRRDRDRDDLDRDDDLDDRRRDRRUUUUULUURUURRRDRRDLDLDDRDDRDDRRRDRURUURURRUULUURURUUULUURRRDDDDDDRDRRUURRRURRURRDDRDRDDRDLDLLLULLUULLDDLDDLDLDLLLLDDDDDDDLDRDRDDRRRUURRULURRURRRDRDDRRDDLLLDLDLLULLDLDDRRDRRDDRDRDDRRDRDDLLLULULDLDDDLLDLUULLDDDLDLULLUURULUULUUULULULLUULLDDLDDLDDRDRRDRDLLLDDDRRRURDRRDRDRDRDRRRRRRUUURRDDRDRRUUUULLURURRRRRURRUURUUUULLDLLULUURULLULU


In [5]:
previous_candidates

[(358, 14, 2, 25915, 'D')]

In [4]:
candidates

[]

In [6]:
aaaa = [(1,3),(2,5),(3,1)]

In [10]:
sorted(aaaa, key=lambda x: -x[1])

[(2, 5), (1, 3), (3, 1)]

In [3]:
"""
次は何をすべき？
スコアを上げるやり方？
高速化?
取りえずソートをrandomではなく、スコアが高い方から先にやる。
"""


import time
start_time = time.time()

N=50

def row_text_to_ints(row):
    return [int(x) for x in row.split()]

fuchi = [-1 for i in range(N+2)]

def add_fuchi(hoges_list):
    hoges_list = [[-1] + hoges + [-1] for hoges in hoges_list]
    hoges_list = [fuchi.copy()] + hoges_list + [fuchi.copy()]
    return hoges_list

def get_input():
    SI, SJ = row_text_to_ints(input())
    # 淵に-1を追加
    ts_list = []
    for _ in range(N):
        ts_list.append(row_text_to_ints(input()))
    
    ps_list = []
    for _ in range(N):
        ps_list.append(row_text_to_ints(input()))
        
    return SI, SJ, ts_list, ps_list

# si, sj, ts_list, ps_list = texts_to_stp(texts)
si, sj, ts_list, ps_list = get_input()



ts_list = add_fuchi(ts_list)
ps_list = add_fuchi(ps_list)
si += 1
sj += 1

import random

# 移動できる方にランダムに移動する
# 淵を-1とする
passed = [-1]

passed.append(ts_list[si][sj])

def get_next_tiles(i, j, score):
    ut = ts_list[i-1][j]
    dt = ts_list[i+1][j]
    lt = ts_list[i][j-1]
    rt = ts_list[i][j+1]

    # tile, i, j をタプルで保存
    tiles = []
    if ut not in passed:
        tiles.append((ut, i-1, j, score +ps_list[i-1][j], "U"))
    if dt not in passed:
        tiles.append((dt, i+1, j, score +ps_list[i+1][j],  "D"))
    if lt not in passed:
        tiles.append((lt, i, j-1,  score +ps_list[i][j-1], "L"))
    if rt not in passed:
        tiles.append((rt, i, j+1,  score +ps_list[i][j+1], "R"))
    # 候補をスコアが高い順でソート
    if tiles:
        sorted(tiles, key=lambda x: -x[2])
    return tiles

best_tiles = []
# taileID, i, j, その時点でのポイント、コマンドを保存する。最初はコマンドなし
best_tiles.append((ts_list[si][sj],si, sj, ps_list[si][sj], ""))
                  
not_passed_candidates_list = []
now_i, now_j = si, sj
now_score = ps_list[now_i][now_j]
max_score = 0
max_score_commands = ""
commands=""
is_end = False


while True:
#     print("while start")
    elapsed_time = time.time() - start_time
    if elapsed_time > 1.9:
        break
        
    candidates = get_next_tiles(now_i, now_j, now_score)
#     print("now_score", now_score)
    if len(candidates) == 0:
#         print("戻る")
        # 行き先がないため戻る
        # この時点でのスコアがこれまでで最大なら、スコアとコマンドを保存しておく。
        if now_score > max_score:
            max_score = now_score
            max_score_commands = commands
        
        # 戻る際には、他の候補がありえたところまで戻る
        while len(not_passed_candidates_list) >0:
            elapsed_time = time.time() - start_time
            if elapsed_time > 1.9:
                is_end=True
                break

            # 戻る際には、 passed,  commandを修正する必要。
            # score、位置はcandidatesに保存されておりそちらを参照するため、修正しなくて良い。
            passed = passed[:-1]
            commands = commands[:-1]
            
            # 最後尾を取り出す
            previous_candidates = not_passed_candidates_list.pop()
#             print("pc",previous_candidates)
            if len(previous_candidates) > 0:
                # 候補があったらそれらをcandidatesにして元の動作を始める
                candidates = previous_candidates
                break
            # candidatesもなく、not_passed_candidates_listが尽きたなら、最大のやつをprintして終了。
            # TODO: 怪しい
            if len(not_passed_candidates_list) == 0:
                is_end = True
                
            # 1つ前に他の候補がなければもう1つ前に戻るしかない
#             print("continue")
            continue
            
    if is_end:
        break

    # candidatesの1つ目以降は行かないリスト。後で行くかも。
    not_passed_candidates_list.append(candidates[1:])

    # 次行く先はcandidatesの先頭
    next_tile = candidates[0]
    passed.append(next_tile[0])
    now_i, now_j = next_tile[1], next_tile[2]
    now_score = next_tile[3]
    commands += next_tile[4]

# print("last")
print(max_score_commands)

UULULUUUURRRDDDDRDDDDLLLDLLUUUULDLUULULULLULLULLLDDDLDLDLUUUULDLLUULDLDDLDDLDDDDDDDDDDDDDLDLDDDDDDRRUURUUUURURURURURRURUUUULLDLLUULLUUURURRDDDRURRRURDDRDDDDDDDDDDDLDLLDDDDDLLDDDDDDDDLULLUUURUULLDLDLDDDLDLDDDLDDRRURURURRDRDDDDLDDLDDLDDDDRURUUURRUUURURUUUUUUURUURUUUUURUUUURDDRDDRRDRUUUUUUULLUUUUUURURUURRUURRDDRDDDDDDDDDDDDDLDDDDDDDLLUULDDDLDDDDLDDDDDDDDDDRRDRUUUURUUURUUUUUURUURURDDDDRDDDDDDDRDDDDLDDDRRRURURURURDRDRURURRURUUUULLDLUUUULUUUULUUUULLDDLULDLUUUUUUULLULUUURURURURUUUUUUURUUUUUURRDDDRURRDDLDDDDDDDLLDDLDDDDDDDRDRDDRDRDRDRURDRURDRDDDDDDDDRRUURRDDDDRDLLLULLDLDDRRRDRRDRRDLLLLULDDLDDLLULUUU
